In [1]:
VKing=1000
VQueen=100
VBishop=40
VKnight=45
VRook=60
VPawn=10

In [2]:
def abs_(x):
    if(x>0):
        return x
    return -x
class Color(object):
    EMPTY=0
    BLACK=1
    WHITE=2
class HowCasting(object):
    NONE=1
    LEFT=2
    RIGHT=3
    ALL=6
class Piece(object):
    IMG= None
    NAME=None
    STRAIGHT_LINE = [[1,0],[-1,0],[0,1],[0,-1]]
    DIAGONAL_LINE = [[1,1],[-1,1],[1,-1],[-1,-1]]
    
    @staticmethod
    def count_step(x,x_):
        if(x_>0):
            return 7-x
        return x

    def __init__(self,x,y,color,value):
        self.color=color
        self.x=x
        self.y=y
        self.value=value

    def get_possible_line_moves(self, board, how_line, step=8):
        enemy_color=Color.BLACK+Color.WHITE-self.color
        legal_moves=[]
        for x_,y_ in how_line:
            for i in range(1,min(step,min(self.count_step(self.x,x_),self.count_step(self.y,y_)))+1):
                if(board[self.y+y_*i][self.x+x_*i].color==Color.EMPTY):
                    legal_moves.append([self.x+x_*i,self.y+y_*i])
                else:
                    if (board[self.y+y_*i][self.x+x_*i].color==enemy_color):
                        legal_moves.append([self.x+x_*i,self.y+y_*i])
                    break
        return legal_moves

    def control_straight_line(self,board,x_,y_):
        dist_x=abs_(self.x-x_)
        dist_y=abs_(self.y-y_)
        if(min(dist_x,dist_y)!=0):
            return False
        if(dist_x==dist_y):
            return True
        if(dist_x==0):
            direction_y=(y_-self.y)//dist_y
            for i in range(1,dist_y):
                if(board[self.y+direction_y*i][x_].NAME!='Empty'):
                    return False
        else:
            direction_x=(x_-self.x)//dist_x
            for i in range(1,dist_x):
                if(board[y_][self.x+direction_x*i].NAME!='Empty'):
                    return False
        return True
    def control_diagonal_line(self,board,x_,y_):
        dist_x=abs_(self.x-x_)
        dist_y=abs_(self.y-y_)
        if(dist_x!=dist_y):
            return False
        if(dist_x==0):
            return True
        direction_x=(x_-self.x)//dist_x
        direction_y=(y_-self.y)//dist_y
        for i in range(1,dist_x):
            if(board[self.y+direction_y*i][self.x+direction_x*i].NAME!='Empty'):
                return False
        return True
        
    def __repr__(self):
        return self.IMG[self.color==Color.BLACK]

    

In [3]:
class Empty(Piece):
    NAME='Empty'
    def __init__(self,x,y,value):
        super().__init__(x,y,Color.EMPTY,value)
    def get_moves(self,board,last_move):
        raise Exception("Error! Здесь нет фигуры!")
    def __repr__(self):
        return '.'
    def clone(self):
        return Empty(self.x,self.y,self.color,self.value)
    
class King(Piece):
    IMG=('♔', '♚')
    NAME='King'
    def __init__(self, x,y,color,value, howCasting_=HowCasting.ALL):
        self.howCasting=howCasting_
        super().__init__(x,y,color,value)
    def get_moves(self,board,last_move):
        legal_moves=self.get_possible_line_moves(board,self.STRAIGHT_LINE,1)+self.get_possible_line_moves(board,self.DIAGONAL_LINE,1)
        #проверка рокировки
        if(self.howCasting%HowCasting.LEFT==0):
            if(board[self.y][1].NAME=='Empty' 
               and board[self.y][2].NAME=='Empty' 
               and board[self.y][3].NAME=='Empty'):
                legal_moves.append([2,self.y])
        if(self.howCasting%HowCasting.RIGHT==0):
            if(board[self.y][6].NAME=='Empty'
               and board[self.y][5].NAME=='Empty'):
                legal_moves.append([6,self.y])
        return legal_moves
    def clone(self):
        return King(self.x,self.y,self.color,self.value)
    
    def IsItControl(self, board,x_,y_):
        dist_x=abs_(self.x-x_)
        dist_y=abs_(self.y-y_)
        return (dist_x<2)+(dist_y<2)==2
class Queen(Piece):
    IMG=('♕', '♛')
    NAME='Queen'
    def get_moves(self,board,last_move):
        legal_moves=self.get_possible_line_moves(board,self.STRAIGHT_LINE)+self.get_possible_line_moves(board,self.DIAGONAL_LINE)
        return legal_moves
    def clone(self):
        return Queen(self.x,self.y,self.color,self.value)
    def IsItControl (self, board, x_,y_):
        return self.control_straight_line(board,x_,y_) or self.control_diagonal_line(board,x_,y_)

class Bishop(Piece):
    IMG=('♗', '♝')
    NAME='Bishop'
    def get_moves(self,board,last_move):
        legal_moves=self.get_possible_line_moves(board,self.DIAGONAL_LINE)
        return legal_moves
    def clone(self):
        return  Bishop(self.x,self.y,self.color,self.value)
    def IsItControl (self,board, x_,y_):
        return self.control_diagonal_line(board,x_,y_)
class Rook(Piece):
    IMG=('♖', '♜')
    NAME='Rook'
    def get_moves(self,board,last_move):
        legal_moves=self.get_possible_line_moves(board,self.STRAIGHT_LINE)
        return legal_moves
    def clone(self):
        return Rook(self.x,self.y,self.color,self.value)
    def IsItControl (self, board, x_,y_):
        return self.control_straight_line(board,x_,y_)
class Knight(Piece):
    IMG=('♘', '♞')
    NAME='Knight'
    def get_moves(self,board,last_move):
        legal_moves=[]
        enemy_color=Color.BLACK+Color.WHITE-self.color
        for x_,y_ in self.DIAGONAL_LINE:
            if(self.y+2*y_<=7 and self.y+2*y_>=0 and self.x+x_<8 and self.x+x_>=0):
                if(board[self.y+2*y_][self.x+x_].color!=self.color):
                    legal_moves.append([self.x+x_,self.y+2*y_])
            if(self.y+y_<=7 and self.y+y_>=0 and self.x+2*x_<8 and self.x+2*x_>=0):
                if(board[self.y+y_][self.x+2*x_].color!=self.color):
                    legal_moves.append([self.x+2*x_,self.y+y_])
        return legal_moves
    def clone(self):
        return Knight(self.x,self.y,self.color,self.value)
    def IsItControl (self, board,x_,y_):
        dist_x=abs_(self.x-x_)
        dist_y=abs_(self.y-y_)
        return (dist_x==2 and dist_y==1) or (dist_x==1 and dist_y==2)
class Pawn(Piece):
    IMG=('♙', '♟')
    NAME='Pawn'

        
    def get_moves(self,board,last_move):
        legal_moves=[]
        enemy_color=Color.BLACK+Color.WHITE-self.color

        ##ход по диагонали, если стоит пешка противника
        if(self.x+1<7 and board[self.y+self.color-enemy_color][self.x+1].color==enemy_color):
            legal_moves.append([self.x+1,self.y+self.color-enemy_color])
        if(self.x-1>=0 and board[self.y+self.color-enemy_color][self.x-1].color==enemy_color):
            legal_moves.append([self.x-1,self.y+self.color-enemy_color])
        ##ход вперед на 1 клетку, если там свободна
        if(board[self.y+self.color-enemy_color][self.x].color==Color.EMPTY):
            legal_moves.append([self.x,self.y+self.color-enemy_color])
        ##ход на 2 клетки вперед, если в этих двух клетках свободна и пешка находится в начале
        if(self.y==1+5*(self.color==Color.BLACK)):
            if(board[self.y+(self.color-enemy_color)*2][self.x].color==Color.EMPTY 
               and board[self.y+self.color-enemy_color][self.x].color==Color.EMPTY):
                legal_moves.append([self.x,self.y+(self.color-enemy_color)*2])
        ##взятие на проходе
        if(last_move is not None):
            last_xy_from=last_move[0] #для этого мы должны знать последний ход противника
            last_xy_to=last_move[1] #это должен быть ход пешкой на 2 клетки вперед
                                    #тогда ее можно подрезать
            if(self.y==3+(self.color==Color.WHITE) and last_xy_to[1]==self.y and abs_(self.x-last_xy_to[0])==1 
            and board[last_xy_to[1]][last_xy_to[0]].NAME=='Pawn' ):
                legal_moves.append([last_xy_to[0],self.y+self.color-enemy_color])
                
        return legal_moves
    def clone(self):
        return Pawn(self.x,self.y,self.color,self.value)
    def IsItControl (self,board, x_,y_):
        dist_x=abs_(self.x-x_)
        enemy_color=Color.BLACK+Color.WHITE-self.color
        return dist_x==1 and y_==self.y+self.color-enemy_color
        

In [20]:
class Board(object):
    ##тут нужна функция предглагающая возможные ходы
    ##проверка мата, это когда королю шах и он не может выйти из шаха
    ##проверка пата, нет возможных ходов и но у короля нет шаха
    
    def __init__(self,chesspieces_=None, whose_move_=Color.WHITE, last_move_=None):
        
        self.whose_move=whose_move_#чей ход
        self.last_move=last_move_#последний ход
        self.king={}
        if(chesspieces_):#задана расстановка фигур
            self.chesspieces=chesspieces_#сохраняем расстановку фигур
        else:#иначе создаем расстановку
            self.chesspieces={#может тут добавить информацию о состоянии рокировки?
                Color.WHITE:[King(4,0,Color.WHITE,VKing), Queen(3,0,Color.WHITE,VQueen),
                             Knight(1,0,Color.WHITE,VKnight),Knight(6,0,Color.WHITE,VKnight),
                             Bishop(2,0,Color.WHITE,VBishop),Bishop(5,0,Color.WHITE,VBishop), 
                             Rook(0,0,Color.WHITE,VRook),Rook(7,0,Color.WHITE,VRook)] +[Pawn(x,1,Color.WHITE,VPawn) for x in range(8)],
                Color.BLACK:[King(4,7,Color.BLACK,VKing), Queen(3,7,Color.BLACK,VQueen),
                             Knight(1,7,Color.BLACK,VKnight),Knight(6,7,Color.BLACK,VKnight),
                            Bishop(2,7,Color.BLACK,VBishop),Bishop(5,7,Color.BLACK,VBishop), 
                            Rook(0,7,Color.BLACK,VRook),Rook(7,7,Color.BLACK,VRook)]+[Pawn(x,6,Color.BLACK,VPawn) for x in range(8)]
            }
        for piece in self.chesspieces[Color.WHITE]:
            if(piece.NAME=='King'):
                self.king[Color.WHITE]=[piece.x,piece.y]
                break
        for piece in self.chesspieces[Color.BLACK]:
            if(piece.NAME=='King'):
                self.king[Color.BLACK]=[piece.x,piece.y]
                break
                
        self.board=Board.init_board(self.chesspieces)#создаем доску по расстановке
        
    @staticmethod
    def init_board(chesspieces):## иницилизация доски с помощью разметки
        board=[[Empty(x,y,0) for x in range(8)] for y in range(8)] ## создаем доску 8 на 8
        Board.init_board_for_one_color(board, chesspieces[Color.WHITE]) #иницилизируем белые фигуры
        Board.init_board_for_one_color(board, chesspieces[Color.BLACK]) #иницилизируем черные фигуры
        return board #возвращаем доску
    @staticmethod
    def init_board_for_one_color(board, chesspieces): #иницилизируем доску определенного цвета
        for piece in  chesspieces:
            if(board[piece.y][piece.x].NAME=='Empty'):
                board[piece.y][piece.x]=piece
            else:
                raise Exception("Error! Не правильная расстановка!")
    def __repr__(self):
        res=''
        for line in self.board:
            res=''.join(map(str,line))+'\n'+res         
        return res
    def get_moves(self,x,y):
        return self.board[y][x].get_moves(self.board,self.last_move)
    def get_legal_moves(self):
        legal_moves=[]
        for piece in self.chesspieces[self.whose_move]:
            moves=piece.get_moves(self.board,self.last_move)
            for move in moves:
                if(piece.NAME=='King'):
                    king=move
                else:
                    king=self.king[self.whose_move]
                copy=Board.clone(self)
                copy.move([piece.x,piece.y],move)
                NotCheck=True
                for piece_copy in copy.chesspieces[copy.whose_move]:
                    if(piece_copy.IsItControl(self.board,king[0],king[1])):
                        NotCheck=False
                        break
                if(NotCheck):
                    legal_moves.append([[piece.x,piece.y],move])
        return legal_moves
    def move(self, xy_from, xy_to):
        #тут должны быть проверки!!!
        if(self.board[xy_from[1]][xy_from[0]].color!=self.whose_move):
            raise Exception("Error! Не Ваш ход!")
        
        enemy_color=Color.WHITE+Color.BLACK-self.whose_move
        legal_moves=self.get_moves(xy_from[0],xy_from[1])
        if xy_to in legal_moves:
            
            #было ли взятие на проходе?
            if(self.board[xy_from[1]][xy_from[0]].NAME=='Pawn' and abs_(xy_from[0]-xy_to[0])==1
               and self.board[xy_to[1]][xy_to[0]].NAME=='Empty'):
                #нужно удалить ту пешку из доски и расстановки фигур
                pawn=self.board[self.last_move[1][1]][self.last_move[1][0]]
                self.chesspieces[enemy_color].remove(pawn)#? не знаю насколько это корректно, нужно затестить
                self.board[pawn.y][pawn.x]=Empty(pawn.x,pawn.y,0)
                
            #произошла рокировка?
            #проверяем что фигура король и возможна ли рокировка
            if(self.board[xy_from[1]][xy_from[0]].NAME=='King' 
               and self.board[xy_from[1]][xy_from[0]].howCasting>HowCasting.NONE):
                if(xy_from[1]==xy_to[1] and (xy_to[0]==2 or xy_to[0]==6)):
                    self.board[xy_from[1]][xy_from[0]].howCasting=HowCasting.NONE
                    #нужно переместить ладью
                    if(xy_to[0]==2):
                        piece=self.board[xy_from[1]][0]
                        piece.x=3
                        self.board[xy_from[1]][0]=Empty(0,xy_from[1],0)
                        self.board[xy_from[1]][3]=piece
                    else:
                        piece=self.board[xy_from[1]][7]
                        piece.x=5
                        self.board[xy_from[1]][7]=Empty(7,xy_from[1],0)
                        self.board[xy_from[1]][5]=piece
                else:
                    self.board[xy_from[1]][xy_from[0]].howCasting=HowCasting.NONE
            #мы ходим ладьей? если да, то если возможность у короля рокироваться?
            piece_king=self.board[self.king[self.whose_move][1]][self.king[self.whose_move][0]]
            if(self.board[xy_from[1]][xy_from[0]].NAME=='Rook' and 
               piece_king.howCasting>HowCasting.NONE):
                
                if(piece_king.howCasting%HowCasting.LEFT==0 and xy_from[0]==0):
                    piece_king.howCasting/=HowCasting.LEFT
                elif(piece_king.howCasting%HowCasting.RIGHT==0 and xy_from[0]==7):
                    piece_king.howCasting/=HowCasting.RIGHT
            #
            #мы съели фигуру противника?
            if(self.board[xy_to[1]][xy_to[0]].color==enemy_color):
                piece=self.board[xy_to[1]][xy_to[0]]
                self.chesspieces[enemy_color].remove(piece)#удаляем из расстаноки фигур
            #теперь реализуем ход
            piece=self.board[xy_from[1]][xy_from[0]]
            self.chesspieces[self.whose_move].remove(piece)
            self.board[xy_from[1]][xy_from[0]]=Empty(piece.x,piece.y,0)
            piece.x=xy_to[0]
            piece.y=xy_to[1]
            self.chesspieces[self.whose_move].append(piece)
            self.board[xy_to[1]][xy_to[0]]=piece

            self.whose_move=enemy_color
            self.last_move=[xy_from, xy_to]
        else:
            raise Exception("Error! Ход не корректен!")
    def get_moves(self,x,y):
        return self.board[y][x].get_moves(self.board,self.last_move)
    def scores(self, color):
        scores=0
        for piece in self.chesspieces[color]:
            if(piece.NAME=='Pawn'):
                count_pawn_on_line=0
                for y_ in range(1,7):
                    if(self.board[y_][piece.x].NAME=='Pawn' and self.board[y_][piece.x].color==color):
                        count_pawn_on_line+=1 
                position_pawn=piece.y*10
                if(color==Color.BLACK):
                    position_pawn=(7-piece.y)*10
                if(piece.x==0 or piece.x==7):
                    scores+=position_pawn/(2*count_pawn_on_line)
                else:
                    count_neigbor=0
                    if((self.board[piece.y-1][piece.x-1].NAME=='Pawn' and self.board[piece.y-1][piece.x-1].color==color)
                        or (self.board[piece.y][piece.x-1].NAME=='Pawn'and self.board[piece.y][piece.x-1].color==color)
                        or (self.board[piece.y+1][piece.x-1].NAME=='Pawn'and self.board[piece.y+1][piece.x-1].color==color)):
                        count_neigbor+=1
                    if((self.board[piece.y-1][piece.x+1].NAME=='Pawn' and self.board[piece.y-1][piece.x+1].color==color)
                        or (self.board[piece.y][piece.x+1].NAME=='Pawn' and self.board[piece.y][piece.x+1].color==color)
                        or (self.board[piece.y+1][piece.x+1].NAME=='Pawn' and self.board[piece.y+1][piece.x+1].color==color)):
                        count_neigbor+=1
                        
                    scores+=position_pawn*(1+count_neigbor/8.)/count_pawn_on_line
            elif(piece.NAME=='King'):
                scores+=900
            else:#Bishop, Knight,Rook, Queen
                #здесь важно расположение фигур друг относительно друга
                moves=piece.get_moves(self.board,self.last_move)
                scores+=5*len(moves)+piece.value
    
        return scores
    def get_Name(self,x,y):
        return self.board[y][x].get_Name()

    @classmethod
    def clone(cls, board):
        chesspieces_white=[piece.clone() for piece in board.chesspieces[Color.WHITE]]
        chesspieces_black=[piece.clone() for piece in board.chesspieces[Color.BLACK]]
        chesspieces={Color.WHITE:chesspieces_white, Color.BLACK:chesspieces_black}
        return cls(chesspieces, board.whose_move, board.last_move)
    @classmethod
    def new(cls):
        return cls()

In [5]:
6%2

0

In [6]:
Board()

♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
........
........
........
........
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖

In [7]:
def game(board):
    best_move=0
    best_score=-1000
    legal_moves=board.get_legal_moves()
    print("-"*40)
    if(legal_moves):
        for move in board.get_legal_moves():
            color=board.whose_move
            copy=Board.clone(board)
            copy.move(move[0],move[1])
            enemy_color=copy.whose_move
            score=copy.scores(color)-copy.scores(enemy_color)
            if(score>best_score):
                best_score=score
                best_move=move
            print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))
    else:
        #либо пат, либо мат
        copy=Board.clone(board)
        king=[0,0]
        for piece in copy.chesspieces[copy.whose_move]:
            if(piece.NAME=='King'):
                king=[piece.x,piece.y]
                break
        copy.whose_move=Color.WHITE+Color.BLACK-copy.whose_move
        isItCheckMate=False
        for piece in copy.chesspieces[copy.whose_move]:
            if(piece.IsItControl (copy.board, king[0],king[1])):
                isItCheckMate=True
                break
        if(isItCheckMate):
            if(copy.whose_move==Color.WHITE):
                text="белые."
            else:
                text="черные."
            print("Шах и мат. Выиграли ",text)
        else:
            print("Пат")

In [8]:
game(Board())

----------------------------------------
[[1, 0], [2, 2]]  scoresW= 1410.0  scoresB= 1395.0
[[1, 0], [0, 2]]  scoresW= 1400.0  scoresB= 1395.0
[[6, 0], [7, 2]]  scoresW= 1400.0  scoresB= 1395.0
[[6, 0], [5, 2]]  scoresW= 1410.0  scoresB= 1395.0
[[0, 1], [0, 2]]  scoresW= 1395.0  scoresB= 1395.0
[[0, 1], [0, 3]]  scoresW= 1403.75  scoresB= 1395.0
[[1, 1], [1, 2]]  scoresW= 1417.5  scoresB= 1395.0
[[1, 1], [1, 3]]  scoresW= 1421.25  scoresB= 1395.0
[[2, 1], [2, 2]]  scoresW= 1417.5  scoresB= 1395.0
[[2, 1], [2, 3]]  scoresW= 1425.0  scoresB= 1395.0
[[3, 1], [3, 2]]  scoresW= 1442.5  scoresB= 1395.0
[[3, 1], [3, 3]]  scoresW= 1450.0  scoresB= 1395.0
[[4, 1], [4, 2]]  scoresW= 1457.5  scoresB= 1395.0
[[4, 1], [4, 3]]  scoresW= 1460.0  scoresB= 1395.0
[[5, 1], [5, 2]]  scoresW= 1402.5  scoresB= 1395.0
[[5, 1], [5, 3]]  scoresW= 1410.0  scoresB= 1395.0
[[6, 1], [6, 2]]  scoresW= 1417.5  scoresB= 1395.0
[[6, 1], [6, 3]]  scoresW= 1421.25  scoresB= 1395.0
[[7, 1], [7, 2]]  scoresW= 1400.0  sco

In [9]:
b1=Board()

In [10]:
b1

♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
........
........
........
........
♙♙♙♙♙♙♙♙
♖♘♗♕♔♗♘♖

In [11]:
b1.move([4,1],[4,3])
b1.move([4,6],[4,4])
b1.move([5,0],[2,3])
b1.move([5,7],[2,4])
b1.move([3,0],[7,4])
b1.move([6,7],[5,5])

In [12]:
b1

♜♞♝♛♚..♜
♟♟♟♟.♟♟♟
.....♞..
..♝.♟..♕
..♗.♙...
........
♙♙♙♙.♙♙♙
♖♘♗.♔.♘♖

In [13]:
b2=Board.clone(b1)

In [14]:
b2.move([7,4],[5,6])

In [15]:
b2

♜♞♝♛♚..♜
♟♟♟♟.♕♟♟
.....♞..
..♝.♟...
..♗.♙...
........
♙♙♙♙.♙♙♙
♖♘♗.♔.♘♖

In [16]:
b2.board[6][5].IsItControl(b2.board,4,7)

True

In [17]:
game(b2)

check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
check
----------------------------------------
Шах и мат. Выиграли  белые.


In [18]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.WHITE)-copy.scores(Color.BLACK)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))

[[4, 0], [4, 1]]  scoresW= 1500.0  scoresB= 1485.0
[[1, 0], [2, 2]]  scoresW= 1545.0  scoresB= 1485.0
[[1, 0], [0, 2]]  scoresW= 1525.0  scoresB= 1485.0
[[6, 0], [7, 2]]  scoresW= 1520.0  scoresB= 1485.0
[[6, 0], [5, 2]]  scoresW= 1520.0  scoresB= 1485.0
[[6, 0], [4, 1]]  scoresW= 1515.0  scoresB= 1485.0
[[0, 1], [0, 2]]  scoresW= 1530.0  scoresB= 1485.0
[[0, 1], [0, 3]]  scoresW= 1538.75  scoresB= 1485.0
[[1, 1], [1, 2]]  scoresW= 1542.5  scoresB= 1485.0
[[1, 1], [1, 3]]  scoresW= 1551.25  scoresB= 1480.0
[[2, 1], [2, 2]]  scoresW= 1532.5  scoresB= 1485.0
[[3, 1], [3, 2]]  scoresW= 1557.5  scoresB= 1485.0
[[3, 1], [3, 3]]  scoresW= 1580.0  scoresB= 1475.0
[[5, 1], [5, 2]]  scoresW= 1522.5  scoresB= 1490.0
[[5, 1], [5, 3]]  scoresW= 1550.0  scoresB= 1490.0
[[6, 1], [6, 2]]  scoresW= 1537.5  scoresB= 1485.0
[[6, 1], [6, 3]]  scoresW= 1521.25  scoresB= 1485.0
[[7, 1], [7, 2]]  scoresW= 1525.0  scoresB= 1485.0
[[7, 1], [7, 3]]  scoresW= 1533.75  scoresB= 1485.0
[[2, 3], [3, 4]]  scoresW= 

In [19]:
best_score

105.0

In [201]:
best_move

[[4, 1], [4, 3]]

In [202]:
b1.move(best_move[0],best_move[1])

In [203]:
b1

♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
........
........
....♙...
........
♙♙♙♙.♙♙♙
♖♘♗♕♔♗♘♖

In [204]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.BLACK)-copy.scores(Color.WHITE)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))

[[0, 6], [0, 5]]  scoresW= 1460.0  scoresB= 1395.0
[[0, 6], [0, 4]]  scoresW= 1460.0  scoresB= 1403.75
[[1, 6], [1, 5]]  scoresW= 1460.0  scoresB= 1417.5
[[1, 6], [1, 4]]  scoresW= 1455.0  scoresB= 1421.25
[[2, 6], [2, 5]]  scoresW= 1460.0  scoresB= 1417.5
[[2, 6], [2, 4]]  scoresW= 1460.0  scoresB= 1425.0
[[3, 6], [3, 5]]  scoresW= 1460.0  scoresB= 1442.5
[[3, 6], [3, 4]]  scoresW= 1460.0  scoresB= 1450.0
[[4, 6], [4, 5]]  scoresW= 1460.0  scoresB= 1457.5
[[4, 6], [4, 4]]  scoresW= 1460.0  scoresB= 1460.0
[[5, 6], [5, 5]]  scoresW= 1460.0  scoresB= 1402.5
[[5, 6], [5, 4]]  scoresW= 1460.0  scoresB= 1410.0
[[6, 6], [6, 5]]  scoresW= 1460.0  scoresB= 1417.5
[[6, 6], [6, 4]]  scoresW= 1460.0  scoresB= 1421.25
[[7, 6], [7, 5]]  scoresW= 1460.0  scoresB= 1400.0
[[7, 6], [7, 4]]  scoresW= 1460.0  scoresB= 1413.75
[[1, 7], [2, 5]]  scoresW= 1460.0  scoresB= 1415.0
[[1, 7], [0, 5]]  scoresW= 1460.0  scoresB= 1405.0
[[6, 7], [7, 5]]  scoresW= 1460.0  scoresB= 1405.0
[[6, 7], [5, 5]]  scoresW= 

In [205]:
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

best_score= 0.0  best_move [[4, 6], [4, 4]]


♜♞♝♛♚♝♞♜
♟♟♟♟.♟♟♟
........
....♟...
....♙...
........
♙♙♙♙.♙♙♙
♖♘♗♕♔♗♘♖

In [206]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.WHITE)-copy.scores(Color.BLACK)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))

[[0, 1], [0, 2]]  scoresW= 1460.0  scoresB= 1460.0
[[0, 1], [0, 3]]  scoresW= 1468.75  scoresB= 1460.0
[[1, 1], [1, 2]]  scoresW= 1482.5  scoresB= 1460.0
[[1, 1], [1, 3]]  scoresW= 1486.25  scoresB= 1455.0
[[2, 1], [2, 2]]  scoresW= 1482.5  scoresB= 1460.0
[[2, 1], [2, 3]]  scoresW= 1475.0  scoresB= 1460.0
[[3, 1], [3, 2]]  scoresW= 1492.5  scoresB= 1460.0
[[3, 1], [3, 3]]  scoresW= 1525.0  scoresB= 1460.0
[[5, 1], [5, 2]]  scoresW= 1457.5  scoresB= 1460.0
[[5, 1], [5, 3]]  scoresW= 1485.0  scoresB= 1460.0
[[6, 1], [6, 2]]  scoresW= 1482.5  scoresB= 1460.0
[[6, 1], [6, 3]]  scoresW= 1476.25  scoresB= 1460.0
[[7, 1], [7, 2]]  scoresW= 1465.0  scoresB= 1460.0
[[7, 1], [7, 3]]  scoresW= 1478.75  scoresB= 1460.0
[[4, 0], [4, 1]]  scoresW= 1410.0  scoresB= 1460.0
[[3, 0], [4, 1]]  scoresW= 1455.0  scoresB= 1460.0
[[3, 0], [5, 2]]  scoresW= 1495.0  scoresB= 1460.0
[[3, 0], [6, 3]]  scoresW= 1510.0  scoresB= 1460.0
[[3, 0], [7, 4]]  scoresW= 1505.0  scoresB= 1460.0
[[1, 0], [2, 2]]  scoresW= 

In [207]:
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

best_score= 65.0  best_move [[3, 1], [3, 3]]


♜♞♝♛♚♝♞♜
♟♟♟♟.♟♟♟
........
....♟...
...♙♙...
........
♙♙♙..♙♙♙
♖♘♗♕♔♗♘♖

In [208]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.BLACK)-copy.scores(Color.WHITE)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))

[[0, 6], [0, 5]]  scoresW= 1525.0  scoresB= 1460.0
[[0, 6], [0, 4]]  scoresW= 1525.0  scoresB= 1468.75
[[1, 6], [1, 5]]  scoresW= 1525.0  scoresB= 1482.5
[[1, 6], [1, 4]]  scoresW= 1520.0  scoresB= 1486.25
[[2, 6], [2, 5]]  scoresW= 1525.0  scoresB= 1482.5
[[2, 6], [2, 4]]  scoresW= 1525.0  scoresB= 1475.0
[[3, 6], [3, 5]]  scoresW= 1525.0  scoresB= 1492.5
[[3, 6], [3, 4]]  scoresW= 1525.0  scoresB= 1525.0
[[5, 6], [5, 5]]  scoresW= 1525.0  scoresB= 1457.5
[[5, 6], [5, 4]]  scoresW= 1525.0  scoresB= 1485.0
[[6, 6], [6, 5]]  scoresW= 1525.0  scoresB= 1482.5
[[6, 6], [6, 4]]  scoresW= 1520.0  scoresB= 1476.25
[[7, 6], [7, 5]]  scoresW= 1525.0  scoresB= 1465.0
[[7, 6], [7, 4]]  scoresW= 1525.0  scoresB= 1478.75
[[4, 7], [4, 6]]  scoresW= 1525.0  scoresB= 1415.0
[[3, 7], [4, 6]]  scoresW= 1525.0  scoresB= 1455.0
[[3, 7], [5, 5]]  scoresW= 1525.0  scoresB= 1510.0
[[3, 7], [6, 4]]  scoresW= 1520.0  scoresB= 1515.0
[[3, 7], [7, 3]]  scoresW= 1525.0  scoresB= 1505.0
[[1, 7], [2, 5]]  scoresW= 

In [209]:
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

best_score= 0.0  best_move [[3, 6], [3, 4]]


♜♞♝♛♚♝♞♜
♟♟♟..♟♟♟
........
...♟♟...
...♙♙...
........
♙♙♙..♙♙♙
♖♘♗♕♔♗♘♖

In [210]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.WHITE)-copy.scores(Color.BLACK)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))

[[0, 1], [0, 2]]  scoresW= 1525.0  scoresB= 1525.0
[[0, 1], [0, 3]]  scoresW= 1533.75  scoresB= 1525.0
[[1, 1], [1, 2]]  scoresW= 1547.5  scoresB= 1525.0
[[1, 1], [1, 3]]  scoresW= 1551.25  scoresB= 1520.0
[[2, 1], [2, 2]]  scoresW= 1552.5  scoresB= 1525.0
[[2, 1], [2, 3]]  scoresW= 1550.0  scoresB= 1525.0
[[5, 1], [5, 2]]  scoresW= 1522.5  scoresB= 1525.0
[[5, 1], [5, 3]]  scoresW= 1535.0  scoresB= 1525.0
[[6, 1], [6, 2]]  scoresW= 1547.5  scoresB= 1525.0
[[6, 1], [6, 3]]  scoresW= 1541.25  scoresB= 1520.0
[[7, 1], [7, 2]]  scoresW= 1530.0  scoresB= 1525.0
[[7, 1], [7, 3]]  scoresW= 1543.75  scoresB= 1525.0
[[4, 0], [4, 1]]  scoresW= 1475.0  scoresB= 1525.0
[[4, 0], [3, 1]]  scoresW= 1485.0  scoresB= 1525.0
[[3, 0], [3, 1]]  scoresW= 1515.0  scoresB= 1525.0
[[3, 0], [3, 2]]  scoresW= 1525.0  scoresB= 1525.0
[[3, 0], [4, 1]]  scoresW= 1515.0  scoresB= 1525.0
[[3, 0], [5, 2]]  scoresW= 1550.0  scoresB= 1525.0
[[3, 0], [6, 3]]  scoresW= 1570.0  scoresB= 1520.0
[[3, 0], [7, 4]]  scoresW= 

In [211]:
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

best_score= 50.0  best_move [[3, 0], [6, 3]]


♜♞♝♛♚♝♞♜
♟♟♟..♟♟♟
........
...♟♟...
...♙♙.♕.
........
♙♙♙..♙♙♙
♖♘♗.♔♗♘♖

In [212]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.BLACK)-copy.scores(Color.WHITE)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))

[[0, 6], [0, 5]]  scoresW= 1570.0  scoresB= 1520.0
[[0, 6], [0, 4]]  scoresW= 1570.0  scoresB= 1528.75
[[1, 6], [1, 5]]  scoresW= 1570.0  scoresB= 1542.5
[[1, 6], [1, 4]]  scoresW= 1565.0  scoresB= 1546.25
[[2, 6], [2, 5]]  scoresW= 1570.0  scoresB= 1547.5
[[2, 6], [2, 4]]  scoresW= 1570.0  scoresB= 1545.0
[[5, 6], [5, 5]]  scoresW= 1570.0  scoresB= 1517.5
[[5, 6], [5, 4]]  scoresW= 1555.0  scoresB= 1535.0
[[6, 6], [6, 5]]  scoresW= 1565.0  scoresB= 1542.5
[[6, 6], [6, 4]]  scoresW= 1555.0  scoresB= 1536.25
[[7, 6], [7, 5]]  scoresW= 1570.0  scoresB= 1525.0
[[7, 6], [7, 4]]  scoresW= 1570.0  scoresB= 1538.75
[[4, 7], [4, 6]]  scoresW= 1570.0  scoresB= 1475.0
[[4, 7], [3, 6]]  scoresW= 1565.0  scoresB= 1490.0
[[3, 7], [3, 6]]  scoresW= 1565.0  scoresB= 1510.0
[[3, 7], [3, 5]]  scoresW= 1570.0  scoresB= 1535.0
[[3, 7], [4, 6]]  scoresW= 1570.0  scoresB= 1510.0
[[3, 7], [5, 5]]  scoresW= 1570.0  scoresB= 1560.0
[[3, 7], [6, 4]]  scoresW= 1555.0  scoresB= 1555.0
[[3, 7], [7, 3]]  scoresW= 

In [213]:
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

best_score= 155.0  best_move [[2, 7], [6, 3]]


♜♞.♛♚♝♞♜
♟♟♟..♟♟♟
........
...♟♟...
...♙♙.♝.
........
♙♙♙..♙♙♙
♖♘♗.♔♗♘♖

In [214]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.WHITE)-copy.scores(Color.BLACK)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

[[0, 1], [0, 2]]  scoresW= 1395.0  scoresB= 1550.0
[[0, 1], [0, 3]]  scoresW= 1403.75  scoresB= 1550.0
[[1, 1], [1, 2]]  scoresW= 1417.5  scoresB= 1550.0
[[1, 1], [1, 3]]  scoresW= 1421.25  scoresB= 1545.0
[[2, 1], [2, 2]]  scoresW= 1407.5  scoresB= 1550.0
[[2, 1], [2, 3]]  scoresW= 1405.0  scoresB= 1550.0
[[5, 1], [5, 2]]  scoresW= 1407.5  scoresB= 1540.0
[[5, 1], [5, 3]]  scoresW= 1405.0  scoresB= 1550.0
[[6, 1], [6, 2]]  scoresW= 1417.5  scoresB= 1550.0
[[7, 1], [7, 2]]  scoresW= 1400.0  scoresB= 1550.0
[[7, 1], [7, 3]]  scoresW= 1413.75  scoresB= 1550.0
[[4, 0], [4, 1]]  scoresW= 1365.0  scoresB= 1545.0
[[4, 0], [3, 1]]  scoresW= 1365.0  scoresB= 1550.0
[[1, 0], [2, 2]]  scoresW= 1410.0  scoresB= 1550.0
[[1, 0], [3, 1]]  scoresW= 1375.0  scoresB= 1550.0
[[1, 0], [0, 2]]  scoresW= 1395.0  scoresB= 1550.0
[[6, 0], [7, 2]]  scoresW= 1395.0  scoresB= 1550.0
[[6, 0], [5, 2]]  scoresW= 1405.0  scoresB= 1540.0
[[6, 0], [4, 1]]  scoresW= 1375.0  scoresB= 1545.0
[[2, 0], [3, 1]]  scoresW= 1

♜♞.♛♚♝♞♜
♟♟♟..♟♟♟
........
...♟♟...
.♙.♙♙.♝.
........
♙.♙..♙♙♙
♖♘♗.♔♗♘♖

In [215]:
best_move=0
best_score=-1000
for move in b1.get_legal_moves():
    copy=Board.clone(b1)
    copy.move(move[0],move[1])
    score=copy.scores(Color.BLACK)-copy.scores(Color.WHITE)
    if(score>best_score):
        best_score=score
        best_move=move
    print(move, ' scoresW=', copy.scores(Color.WHITE), ' scoresB=', copy.scores(Color.BLACK))
print("best_score=",best_score," best_move", best_move)
b1.move(best_move[0],best_move[1])
b1

[[0, 6], [0, 5]]  scoresW= 1421.25  scoresB= 1545.0
[[0, 6], [0, 4]]  scoresW= 1421.25  scoresB= 1553.75
[[1, 6], [1, 5]]  scoresW= 1421.25  scoresB= 1557.5
[[1, 6], [1, 4]]  scoresW= 1416.25  scoresB= 1561.25
[[2, 6], [2, 5]]  scoresW= 1421.25  scoresB= 1572.5
[[2, 6], [2, 4]]  scoresW= 1421.25  scoresB= 1575.0
[[5, 6], [5, 5]]  scoresW= 1421.25  scoresB= 1542.5
[[5, 6], [5, 4]]  scoresW= 1421.25  scoresB= 1550.0
[[6, 6], [6, 5]]  scoresW= 1421.25  scoresB= 1567.5
[[6, 6], [6, 4]]  scoresW= 1416.25  scoresB= 1561.25
[[7, 6], [7, 5]]  scoresW= 1421.25  scoresB= 1550.0
[[7, 6], [7, 4]]  scoresW= 1421.25  scoresB= 1558.75
[[4, 7], [4, 6]]  scoresW= 1421.25  scoresB= 1505.0
[[4, 7], [3, 6]]  scoresW= 1421.25  scoresB= 1525.0
[[3, 7], [2, 7]]  scoresW= 1421.25  scoresB= 1525.0
[[3, 7], [3, 6]]  scoresW= 1421.25  scoresB= 1540.0
[[3, 7], [3, 5]]  scoresW= 1421.25  scoresB= 1555.0
[[3, 7], [4, 6]]  scoresW= 1421.25  scoresB= 1530.0
[[3, 7], [5, 5]]  scoresW= 1421.25  scoresB= 1580.0
[[3, 7],

♜♞.♛♚.♞♜
♟♟♟..♟♟♟
........
...♟♟...
.♝.♙♙.♝.
........
♙.♙..♙♙♙
♖♘♗.♔♗♘♖

In [94]:
b1.scores(Color.WHITE)

Pawn on  0   1  score= 5.0
count_pawn_on_line= 1
Pawn on  1   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  3   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  4   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  5   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  6   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   1  score= 5.0
count_pawn_on_line= 1
Queen  on  3   0  score= 0
Knight  on  1   0  score= 20
Knight  on  6   0  score= 20
Bishop  on  2   0  score= 0
Bishop  on  5   0  score= 0
Rook  on  0   0  score= 0
Rook  on  7   0  score= 0


1070.0

In [99]:
b1

♜♞♝♛♚♝♞♜
♟♟♟♟♟♟♟♟
........
........
....♙...
........
♙♙♙♙.♙♙♙
♖♘♗♕♔♗♘♖

In [96]:
b1.scores(Color.BLACK)

Pawn on  0   6  score= 5.0
count_pawn_on_line= 1
Pawn on  1   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  3   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  4   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  5   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  6   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   6  score= 5.0
count_pawn_on_line= 1
Queen  on  3   7  score= 0
Knight  on  1   7  score= 20
Knight  on  6   7  score= 20
Bishop  on  2   7  score= 0
Bishop  on  5   7  score= 0
Rook  on  0   7  score= 0
Rook  on  7   7  score= 0


1070.0

In [42]:
l=b1.get_legal_moves()

In [98]:
b1.move([4,1],[4,3])

Exception: Error! Не Ваш ход!

In [100]:
b1.scores(Color.WHITE)

Pawn on  0   1  score= 5.0
count_pawn_on_line= 1
Pawn on  1   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  3   1  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  5   1  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  6   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   1  score= 5.0
count_pawn_on_line= 1
Queen  on  3   0  score= 40
Knight  on  1   0  score= 20
Knight  on  6   0  score= 30
Bishop  on  2   0  score= 0
Bishop  on  5   0  score= 50
Rook  on  0   0  score= 0
Rook  on  7   0  score= 0
Pawn on  4   3  score= 30.0
count_pawn_on_line= 1  count_neigbor= 0


1170.0

In [101]:
b1.scores(Color.BLACK)

Pawn on  0   6  score= 5.0
count_pawn_on_line= 1
Pawn on  1   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  3   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  4   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  5   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  6   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   6  score= 5.0
count_pawn_on_line= 1
Queen  on  3   7  score= 0
Knight  on  1   7  score= 20
Knight  on  6   7  score= 20
Bishop  on  2   7  score= 0
Bishop  on  5   7  score= 0
Rook  on  0   7  score= 0
Rook  on  7   7  score= 0


1070.0

In [102]:
b1.move([3,6],[3,4])
b1.move([4,3],[3,4])
b1.move([4,6],[4,4])

In [103]:
b1

♜♞♝♛♚♝♞♜
♟♟♟..♟♟♟
........
...♙♟...
........
........
♙♙♙♙.♙♙♙
♖♘♗♕♔♗♘♖

In [104]:
b1.scores(Color.WHITE)

Pawn on  0   1  score= 5.0
count_pawn_on_line= 1
Pawn on  1   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  3   1  score= 7.5
count_pawn_on_line= 2  count_neigbor= 1
Pawn on  5   1  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  6   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   1  score= 5.0
count_pawn_on_line= 1
Queen  on  3   0  score= 40
Knight  on  1   0  score= 20
Knight  on  6   0  score= 30
Bishop  on  2   0  score= 0
Bishop  on  5   0  score= 50
Rook  on  0   0  score= 0
Rook  on  7   0  score= 0
Pawn on  3   4  score= 20.0
count_pawn_on_line= 2  count_neigbor= 0


1152.5

In [105]:
b1.scores(Color.BLACK)

Pawn on  0   6  score= 5.0
count_pawn_on_line= 1
Pawn on  1   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   6  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  5   6  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  6   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   6  score= 5.0
count_pawn_on_line= 1
Queen  on  3   7  score= 70
Knight  on  1   7  score= 30
Knight  on  6   7  score= 30
Bishop  on  2   7  score= 50
Bishop  on  5   7  score= 50
Rook  on  0   7  score= 0
Rook  on  7   7  score= 0
Pawn on  4   4  score= 30.0
count_pawn_on_line= 1  count_neigbor= 0


1240.0

In [109]:
b1.scores(Color.WHITE)

Pawn on  0   1  score= 5.0
count_pawn_on_line= 1
Pawn on  1   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  3   1  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  5   1  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  6   1  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   1  score= 5.0
count_pawn_on_line= 1
Queen  on  3   0  score= 40
Knight  on  1   0  score= 20
Knight  on  6   0  score= 30
Bishop  on  2   0  score= 0
Bishop  on  5   0  score= 50
Rook  on  0   0  score= 0
Rook  on  7   0  score= 0
Pawn on  4   5  score= 50.0
count_pawn_on_line= 1  count_neigbor= 0


1190.0

In [110]:
b1.scores(Color.BLACK)

Pawn on  0   6  score= 5.0
count_pawn_on_line= 1
Pawn on  1   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  2   6  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  5   6  score= 15.0
count_pawn_on_line= 1  count_neigbor= 1
Pawn on  6   6  score= 20.0
count_pawn_on_line= 1  count_neigbor= 2
Pawn on  7   6  score= 5.0
count_pawn_on_line= 1
Queen  on  3   7  score= 70
Knight  on  1   7  score= 30
Knight  on  6   7  score= 30
Bishop  on  2   7  score= 20
Bishop  on  5   7  score= 50
Rook  on  0   7  score= 0
Rook  on  7   7  score= 0


1180.0

In [108]:
b1

♜♞♝♛♚♝♞♜
♟♟♟..♟♟♟
....♙...
........
........
........
♙♙♙♙.♙♙♙
♖♘♗♕♔♗♘♖

In [107]:
b1.move([3,4],[4,5])

In [48]:
b1.get_legal_moves()

[[[0, 6], [[0, 5], [0, 4]]],
 [[1, 6], [[1, 5], [1, 4]]],
 [[2, 6], [[2, 5], [2, 4]]],
 [[3, 6], [[3, 5], [3, 4]]],
 [[4, 6], [[4, 5], [4, 4]]],
 [[5, 6], [[5, 5], [5, 4]]],
 [[6, 6], [[6, 5], [6, 4]]],
 [[7, 6], [[7, 5], [7, 4]]],
 [[4, 7], []],
 [[3, 7], []],
 [[1, 7], [[2, 5], [0, 5]]],
 [[6, 7], [[7, 5], [5, 5]]],
 [[2, 7], []],
 [[5, 7], []],
 [[0, 7], []],
 [[7, 7], []]]

In [66]:
b1.move([4,6],[4,4])

In [67]:
b1

♜♞♝♛♚♝♞♜
♟♟♟..♟♟♟
........
...♙♟...
........
........
♙♙♙♙.♙♙♙
♖♘♗♕♔♗♘♖

In [68]:
b1.move([3,4],[4,5])

NameError: name 'y' is not defined

In [52]:
b1.get_legal_moves()

[[[0, 1], [[0, 2], [0, 3]]],
 [[1, 1], [[1, 2], [1, 3]]],
 [[2, 1], [[2, 2], [2, 3]]],
 [[3, 1], [[3, 2], [3, 3]]],
 [[5, 1], [[5, 2], [5, 3]]],
 [[6, 1], [[6, 2], [6, 3]]],
 [[7, 1], [[7, 2], [7, 3]]],
 [[4, 0], [[4, 1]]],
 [[3, 0], [[4, 1], [5, 2], [6, 3], [7, 4]]],
 [[1, 0], [[2, 2], [0, 2]]],
 [[6, 0], [[7, 2], [5, 2], [4, 1]]],
 [[2, 0], []],
 [[5, 0], [[4, 1], [3, 2], [2, 3], [1, 4], [0, 5]]],
 [[0, 0], []],
 [[7, 0], []],
 [[4, 3], [[3, 4], [4, 4]]]]